# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 3
REFERENCE_PANEL = "1000G"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: 1000G"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr/gene_corrs-chr3.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/1000g/mashr/by_chr/corrected_positive_definite/gene_corrs-chr3.pkl"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  # eigenvalues <- eigen(data)$values
  # nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  cholStatus <- try(u <- chol(data), silent = FALSE)
  cholError <- ifelse(class(cholStatus) == "try-error", TRUE, FALSE)

  if (cholError) {
    return(FALSE)
  } else {
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 393 393

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000134121,ENSG00000091181,ENSG00000144455,ENSG00000170364,ENSG00000150995,ENSG00000134107,ENSG00000134109,ENSG00000180914,ENSG00000196220,ENSG00000163719
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000134121,1.00000000,0.017674191,0.012542079,0.016677262,0.01847881,0.016097948,0.018822916,0.00000000,0.000000000,0.000000000
ENSG00000091181,0.01767419,1.000000000,0.003377263,0.010511913,0.01021661,0.008805840,0.010159363,0.00000000,0.000000000,0.000000000
ENSG00000144455,0.01254208,0.003377263,1.000000000,0.230119288,0.23898986,0.005771232,0.007381582,0.01151996,0.006743266,0.000000000
ENSG00000170364,0.01667726,0.010511913,0.230119288,1.000000000,0.02445648,0.005700651,0.005812365,0.01076115,0.012330369,0.000000000
ENSG00000150995,0.01847881,0.010216606,0.238989856,0.024456478,1.00000000,0.058599645,0.013710487,0.01485526,0.009410961,0.007316525
ENSG00000134107,0.01609795,0.008805840,0.005771232,0.005700651,0.05859964,1.000000000,0.010874770,0.01345445,0.006885486,0.008300772


# Check positive definiteness

In [17]:
orig_matrix_is_positive_definite <- is_positive_definite(gene_corrs)

In [18]:
if (orig_matrix_is_positive_definite) {
  IRdisplay::display("Matrix is already positive definite!")
  file.copy(INPUT_FILE, OUTPUT_FILE)
} else {
  IRdisplay::display("We need to correct the data and make the matrix positive definite")

  eigenvalues <- eigen(gene_corrs)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
  IRdisplay::display(length(nonpositive_eigenvalues))
  IRdisplay::display(nonpositive_eigenvalues)

  # Make matrix positive definite
  gene_corrs_corrected <- CorrectCM(gene_corrs, 1e-10)
  dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)
  dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)
  gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

  # check if new matrix is really positive definite
  matrix_is_positive_definite <- is_positive_definite(gene_corrs_corrected)
  if (matrix_is_positive_definite) {
    IRdisplay::display("It worked!")
    IRdisplay::display(head(gene_corrs[1:10, 1:10]))
    IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))
    py_save_object(gene_corrs_corrected, OUTPUT_FILE)
  } else {
    eigenvalues <- eigen(gene_corrs_corrected)$values
    nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
    IRdisplay::display(length(nonpositive_eigenvalues))
    IRdisplay::display(nonpositive_eigenvalues)

    stop("Method failed to adjust matrix")
  }
}

[1] "Matrix is already positive definite!"

[1] TRUE

Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).